This kernel is from https://www.kaggle.com/knowledgegrappler/a-keras-prototype-0-21174-on-pl

In [1]:
import numpy as np
np.random.seed(777)
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_json('data/train.json')
test = pd.read_json('data/test.json')

In [3]:
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)

In [4]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis],
                         x_band2[:, :, :, np.newaxis],
                         ((x_band1+x_band2)/2)[:, :, :, np.newaxis]],
                        axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train['is_iceberg'])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])
X_test = np.concatenate([x_band1[: ,:, :, np.newaxis],
                        x_band2[:, :, :, np.newaxis],
                        ((x_band1+x_band2)/2)[:, :, :, np.newaxis]],
                        axis=-1)
X_angle_test = np.array(test.inc_angle)
X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train,
                                                                                   X_angle_train,
                                                                                   y_train,
                                                                                   random_state=123,
                                                                                   train_size=0.75)


In [5]:
X_train.shape

(1203, 75, 75, 3)

In [6]:
X_valid.shape

(401, 75, 75, 3)

In [7]:
X_angle_train.shape

(1203,)

In [8]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

Using TensorFlow backend.


In [9]:
def get_model():
    bn_model = 0
    p_activation = "elu"
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(16, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(32, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = Conv2D(64, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = Conv2D(128, kernel_size = (3,3), activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.2)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, kernel_size = (3,3), activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(0.2)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_concat =  (Concatenate()([img_1, img_2, BatchNormalization(momentum=bn_model)(input_2)]))
    
    dense_layer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_layer = Dropout(0.5) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_layer) ))
    output = Dense(1, activation="sigmoid")(dense_layer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_1 (InputLayer)                (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 75, 75, 3)    12          X_1[0][0]                        
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 73, 73, 16)   448         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 71, 71, 16)   2320        conv2d_1[0][0]                   
____________________________________________________________________________________________

MaxPooling vs Global MaxPooling: https://stats.stackexchange.com/questions/257321/what-is-global-max-pooling-layer-and-what-is-its-advantage-over-maxpooling-layer

In [10]:
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model = get_model()
model.fit([X_train, X_angle_train], y_train, epochs=25
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=32
         , callbacks=callbacks)

Train on 1203 samples, validate on 401 samples
Epoch 1/25
1203/1203 [==============================] - 22s 18ms/step - loss: 0.6434 - accuracy: 0.7016 - val_loss: 0.6523 - val_accuracy: 0.7830
Epoch 2/25
1203/1203 [==============================] - 17s 14ms/step - loss: 0.4770 - accuracy: 0.8088 - val_loss: 0.8516 - val_accuracy: 0.6559
Epoch 3/25
1203/1203 [==============================] - 18s 15ms/step - loss: 0.4138 - accuracy: 0.8238 - val_loss: 0.4541 - val_accuracy: 0.8229
Epoch 4/25
1203/1203 [==============================] - 17s 14ms/step - loss: 0.3881 - accuracy: 0.8396 - val_loss: 0.3852 - val_accuracy: 0.8354
Epoch 5/25
1203/1203 [==============================] - 19s 16ms/step - loss: 0.3635 - accuracy: 0.8504 - val_loss: 0.3940 - val_accuracy: 0.8628
Epoch 6/25
1203/1203 [==============================] - 17s 14ms/step - loss: 0.3723 - accuracy: 0.8429 - val_loss: 0.4122 - val_accuracy: 0.8279
Epoch 7/25
1203/1203 [==============================] - 18s 15ms/step - loss:

In [12]:
model.load_weights(file_path)

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
1203/1203 [==============================] - 3s 2ms/step
[0.20749364752424626, 0.9268495440483093]
####################
watch list evaluate:
401/401 [==============================] - 1s 3ms/step
[0.2700820427174283, 0.8753117322921753]


In [15]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 21s 3ms/step


In [16]:
prediction

array([[1.3150629e-01],
       [3.6513507e-02],
       [1.4275312e-05],
       ...,
       [2.0164308e-01],
       [9.6934515e-01],
       [1.1771917e-05]], dtype=float32)

In [17]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.131506
1,4023181e,0.036514
2,b20200e4,0.000014
3,e7f018bb,0.879145
4,4371c8c3,0.035308
5,a8d9b1fd,0.208136
6,29e7727e,0.262009
7,92a51ffb,0.954778
8,c769ac97,0.129079
9,aee0547d,0.105991


In [18]:
submission.to_csv("./submission.csv", index=False)
